# Forecast Model Run Collections (FMRC) with a custom Xarray Index

There are many ways one might index weather forecast output. These different ways of constructing views of a forecast data are called "Forecast Model Run Collections" (FMRC).
- "Model Run" : a single model run.
- "Constant Offset" : all values for a given lead time.
- "Constant Forecast" : all forecasts for a given time in the future.
- "Best Estimate" : A best guess stitching together the analysis or initialization fields for past forecasts with the latest forecast.

For reference, see [this classic image](https://www.unidata.ucar.edu/presentations/caron/FmrcPoster.pdf).

Assume that a data cube has been constructed with `forecast_reference_time` (commonly `time`) and `forecast_period` (commonly `step` or `lead`) as dimensions.


Then the more complex indexing patterns --- "Constant Forecast" or "Best Estimate" --- are achievable with numpy-style vectorized indexing.
This notebook demonstrates all 4 "FMRC" indexing patterns with a custom Xarray index.


Some complexity arises from models like HRRR where not all runs are the same length (unlike GFS).
This complexity is handled by hardcoding in what data is available for each model: for example, with HRRR we know there are 49 `step`s available every 6 hours, and 19 `steps` otherwise.

In [ ]:
from forecast_index import (
    BestEstimate,
    ConstantForecast,
    ConstantOffset,
    ForecastIndex,
    Model,
    ModelRun,
)

## Create an example data cube

In [ ]:
import numpy as np
import pandas as pd
import xarray as xr

xr.set_options(keep_attrs=True)

ds = xr.Dataset(attrs={"description": "Example HRRR-like dataset"})
shape = {"x": 40, "time": 365 * 24, "step": 49}
ds["foo"] = (("x", "time", "step"), np.ones(tuple(shape.values())))
ds["time"] = (
    "time",
    pd.date_range("2023-01-01", freq="h", periods=shape["time"]),
    {"standard_name": "forecast_reference_time"},
)
ds["step"] = (
    "step",
    pd.to_timedelta(np.arange(0, 49), unit="hours"),
    {"standard_name": "forecast_period"},
)

In [ ]:
ds.chunk(time=1, step=1)

In [ ]:
ds["foo"] = xr.where(
    ~((ds.time.dt.hour % 6 != 0) & (ds.step.dt.total_seconds() / 3600 > 18)),
    ds.foo,
    np.nan,
)
ds.foo.isel(x=20).drop_vars("step").plot(x="time")

In [ ]:
ideal_ds = ds
ideal_ds

In [ ]:
import arraylake as al

In [ ]:
client = al.Client()
repo = client.get_repo("earthmover-demos/hrrr")
ds = repo.to_xarray("solar", chunks={})
ds

## Create a ForecastIndex

In [ ]:
# Add a dummy variable to enable nice indexing syntax
# e.g. `.sel(forecast=ConstantForecast('2024-05-22'))`
ds.coords["forecast"] = 0

# set the new index
newds = (
    ds.isel(time=slice(-120, None))
    .drop_indexes(["time", "step"])
    .set_xindex(["time", "step", "forecast"], ForecastIndex, model=Model.HRRR)
)
newds

## "Standard" selection

In [ ]:
newds.sel(time=slice("2024-05-03", None), step=slice("2h", "12h"))

## FMRC indexing

For all cases, the index knows how to select values so there are no NaNs i.e. it only tries to pull out valid data! 
We've told it this is HRRR, and the HRRR output characteristics are hard-coded in the index. This could be refactored a little.

Other models, like GFS, don't require any configuration.


### `BestEstimate`

In [ ]:
subset = newds.sel(forecast=BestEstimate())
# assert not subset.foo.isnull().any().item()
subset

### `ConstantForecast`

**TODO:** Adding a scalar `valid_time` for `ConstantForecast` triggers a bug.

In [ ]:
subset = newds.sel(forecast=ConstantForecast("2024-06-29"))
# assert not subset.foo.isnull().any().item()
subset

### `ConstantOffset`

In [ ]:
subset = newds.sel(forecast=ConstantOffset("32h"))
# assert not subset.foo.isnull().any().item()
subset

### `ModelRun`

In [ ]:
subset = newds.sel(forecast=ModelRun("2024-06-30 03:00"))  # 19
assert subset.sizes["step"] == 19
subset

In [ ]:
subset = newds.sel(forecast=ModelRun("2024-06-30 06:00"))  # 49
assert subset.sizes["step"] == 49
subset